In [2]:
## change the following to where you have extracted and stored the files
path = '/Users/arrakis/Dropbox/Tool - Classes/ChBE 8803/Project/'

###################################################################

In [4]:
from PIL import Image
import glob
import numpy as np
import pandas as pd
import zipfile,io
import pylab as plt
import matplotlib.image as mpimg

#Edited to allow for grabbing masks and images from zip files

def load_img(path,index,mask_index): #Load an image and masks
    #Lead it to the zip folder
    z=zipfile.ZipFile(path,'r') #Access zip folder
    names=z.namelist() #List of names
    name=names[index] #Get selected image
    name=name[0:-1] #Eliminate "/"
    mask_list=[]
    for string in names:
        if string.startswith(name+'/mask'):
            mask_list.append(string)
    mask_list=mask_list[1:-1] #list of masks
    mask_name=mask_list[mask_index] #Get individual mask
    img_raw=z.read('{}/images/{}.png'.format(name,name)) #Grab raw image
    mask_raw=z.read(mask_name) #Grab a mask
    img = io.BytesIO(img_raw) # convert image
    img = mpimg.imread(img) #numpy array
    img = np.flip(img,0) #flip image
    mask = io.BytesIO(mask_raw) #Convert mask
    mask = mpimg.imread(mask) #numpy array
    mask = np.flip(mask,0) #flip mask too
    
    return img, mask,mask_list

# WORKING EXAMPLE OF LOAD_IMG
bytes_io = load_img(path+'/stage1_train.zip',0,0)
img = mpimg.imread(bytes_io) # numpy array
# imgplot = plt.imshow(img,origin='lower')
# plt.show()
print(img[0][0]*255)

def png2idx(img_path):
    png = Image.open(img_path)
    np_png = np.array(png.getdata())
    df_png = pd.DataFrame(np_png)
    return np_png, df_png

# train_list = glob.glob(path+'stage1_train/*') # ALL TRAINING SAMPLES
train_list = [path+'stage1_train/0a7d30b252359a10fd298b638b90cb9ada3acced4e0c0e5a3692013f432ee4e9'] # 1st training sample, for testing purposes

img_mask_db = pd.DataFrame(columns=['image_ID','mask_ID']) # df containing ID for images and associated masks
for train in train_list:
    # images
    img_id = train.split('train/')[1]
    img_path = train+'/images/'+img_id+'.png'
    np_img, df_img = png2idx(img_path) # returns one-indexed image as np array AND pandas df
    # mask
    mask_list = glob.glob(train+'/masks/*.png')
    mask_id = []
    for mask in mask_list:
        m = mask.split('masks/')[1]
        mask_id = np.append(mask_id, m.split('.png'[0])[0]) # why does there need to be 2 [0]'s ??
        np_mask, df_mask = png2idx(mask) # returns one-indexed mask as np array AND pandas df
    img_mask_db = img_mask_db.append({'image_ID':img_id, 'mask_ID':mask_id}, ignore_index=True)
    
print(df_img.head(1))

[   5.    5.    5.  255.]
   0  1  2    3
0  3  3  3  255
